In [ ]:
! pip install -U sentence-transformers

In [4]:
#Transformando as sentenças em embeddings
from sentence_transformers import SentenceTransformer
sentences = ["Universidade Federal do Piauí", "Processamento de Língua Natural"]
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings.shape) 
similaridade = model.similarity()

(3, 384)
tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

Tomando como base o site da documentação do HugginFace ( https://www.sbert.net/docs/sentence_transformer/pretrained_models.html ) existem vários modelos originais e modelos da comunidade. 

1. Modelos originais


 São modelos que utilizam SetenceTransformer para gerar embeddings gerais de sentenças e consultas. Os modelos da família all-*, treinados em mais de 1 bilhão parametros. Esse modelo all-mpnet-base-v2 mapeia frases e parágrafos para um espaço vetorial de 768 dimensões e pode ser usado para tarefas como agrupamento ou pesquisa semântica.

In [24]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-mpnet-base-v2")
frase_1 = model.encode("O gato está no telhado")
frase_2 = model.encode("Um felino está sobre o teto da casa")
similaridade = util.cos_sim(frase_1, frase_2)
print(f"Similaridade semântica: {similaridade.item()}")

Similaridade semântica: 0.5479925870895386


Os modelos originais são capazes de gerar bons embeddings e no código acima geramos o embedding da frase 1 e da frase 2 e ao final foi feito a similiradide do coseno para buscar a proximidade entre os vatores em busca de uma similaridade entre as fraes. Antes de olhar o resultado podemos ver que apesa das duas frases não terem nenhuma palavra repetida, ou seja, uma palavra de uma frase esta contida na outra frase você pode perceber que o significado permanece o mesmo, ou bem similar, ja que eu apenas troque a palavra mantgendo o singificado, ou seja, eu fiz a substituição de palavras por sinônimos, em vez de "gato" utilizei "felino", em vez de "telhado" eu utilizei "teto da casa". Ao visualizar o resutlado de similaridade um resultado de 0.54 em uma escala que vai de -1 (sentido oposto), passando por 0 (não tem relação) até 1 que seria exatamente similar, ou seja, mesma frase, podemos perceber que não foi um resutlado satisfatório. Ao visualilzar detalhes do modelo temos que ele tem modelo base microsoft/mpnet-base, tem uma dimensão de 768, ou seja, cada vetor de embedding tem 768 valores, utiilzou tecnica de agregação chamda mean pooling e foi treina com mais de 1 bilhão de parametros, então era para dar um resultado melhor. Ao entrar nesse link do [modelo](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) podemos ver os datasets que foi utilizado como treino, como a base do reddit, citacoes do s2orc, respostas do wikipedia, entre outros. Assim a base utilizada foi o ingles, então vamos alterar o código e utilizar frases em ingles:

In [26]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-mpnet-base-v2")

frase_1 = model.encode("The cat is on the roof")
frase_2 = model.encode("A feline is on top of the house roof")

similaridade = util.cos_sim(frase_1, frase_2)
print(f"Similaridade semântica: {similaridade.item()}")

Similaridade semântica: 0.8366097807884216


Podemos perceber que o valor agora foi para 0.83 que é próximo de 1, agora faz sentido!!!

2. Semantic Search Models

Modelos otimizados para busca semântica, projetados para encontrar passagens relevantes com base em uma consulta. O base model são modelos gerais que não foram ajustados para tarefas específicas, foram treinados de forma ampla para aprender representações de linguagem natural, como a relação contextual entre palavras e frases, ja os modelos de Semantic Search são modelos ajustados especificamente para tarefas de busca semântica, onde o objetivo é encontrar passagens de texto relevantes em resposta a uma consulta. Exemplos de uso: Sistemas de recuperação de documentos, FAQs, busca de respostas em grandes bancos de dados textuais.

Vamos simular um FAQ com o model multi-qa-mpnet-base-cos-v1

In [30]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("multi-qa-mpnet-base-cos-v1")

consulta = "Esqueci minha senhja, como posso recuperar?"
embedding_consulta = model.encode(consulta)

faq_perguntas = [
    "Como faço para redefinir minha senha?",
    "Quais métodos de pagamento são aceitos?",
    "Como posso rastrear meu pedido?",
    "Qual é o horário de atendimento ao cliente?",
    "Vocês oferecem reembolso?",
]

faq_respostas = [
    "Para redefinir sua senha, acesse as configurações da conta e clique em 'Esqueci minha senha'. Siga as instruções enviadas ao seu e-mail.",
    "Aceitamos pagamentos via cartão de crédito, PayPal e transferência bancária.",
    "Após o envio do pedido, você receberá um e-mail com o código de rastreamento e um link para acompanhar sua entrega.",
    "Nosso atendimento está disponível de segunda a sexta, das 9h às 18h.",
    "Sim, oferecemos reembolso para produtos devolvidos em até 30 dias após a compra, desde que estejam em boas condições."
]

embeddings = model.encode(faq_perguntas)

similarity = model.similarity(embedding_consulta, embeddings)

indice_mais_similar = similarity.argmax()
resposta_encontrada = faq_respostas[indice_mais_similar]

print(f"Pergunta do usuário: {consulta}")
print(f"Resposta mais relevante: {resposta_encontrada}")

Pergunta do usuário: Esqueci minha senhja, como posso recuperar?
Resposta mais relevante: Para redefinir sua senha, acesse as configurações da conta e clique em 'Esqueci minha senha'. Siga as instruções enviadas ao seu e-mail.


Nesse caso é feito o embedding da pergunta (consulta) e o embedding de cada pergunta da FAQ, depois com o similarity ele busca a similaridade com as perguntas do FAQ, como ele retorna o indice da pergunta então utilize esse paga pegar a resposta correspondente. 

3. Multi-QA Models

Os modelos Multi-QA são modelos treinados para realizar buscas sem6anticas usando grandes volumes de dados de perguntas e respotas (QA). 

- multi-qa-mpnet-base-dot-v1 e multi-qa-mpnet-base-cos-v1: Esses modelos utilizam a arquitetura MPNet e fornecem alta precisão, sendo os mais indicados para tarefas em que a qualidade de busca é prioridade.
-	multi-qa-distilbert-dot-v1 e multi-qa-distilbert-cos-v1: Versões menores baseadas em DistilBERT, oferecendo um bom equilíbrio entre desempenho e velocidade.
-	multi-qa-MiniLM-L6-dot-v1 e multi-qa-MiniLM-L6-cos-v1: Esses modelos MiniLM são os mais rápidos em processamento de consultas, ideais para aplicações em tempo real que precisam de um alto número de consultas por segundo.

In [32]:
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer("multi-qa-distilbert-dot-v1")

consulta = "Como posso redefinir minha senha?"
query_embedding = model.encode(consulta)

faq_perguntas = [
    "Para redefinir sua senha, vá até as configurações e selecione 'Esqueci minha senha'.",
    "Aceitamos cartão de crédito, PayPal e transferências bancárias.",
    "Você pode rastrear seu pedido através do link de rastreamento fornecido em seu e-mail."
]

passage_embeddings = model.encode(faq_perguntas)

similaridade = util.cos_sim(query_embedding, passage_embeddings)

print("Similaridade entre a consulta e as passagens do FAQ:")
print(similaridade)

Similaridade entre a consulta e as passagens do FAQ:
tensor([[0.6782, 0.4743, 0.3694]])


Utilizei o modelo multi-qa-distilbert-dot-v1 que são versões menores e oferece um bom equilíbrio entre eficiencia e desempenho. Podemos notar que a execução desse código doi levemente mais rápido do que o anterior

Multilingual Models

São modelos projetados para gerar embeddings semelhantes para o mesmo texto em diferentes idiomas, sem a necessidade de especificar o idioma de entrada. Isso significa que, independentemente do idioma, textos que transmitem a mesma mensagem ou significado terão embeddings semelhantes.

In [33]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

frases = [
    "How are you?",           # Inglês
    "¿Cómo estás?",           # Espanhol
    "Comment ça va?",         # Francês
    "Wie geht es dir?",       # Alemão
    "Como você está?"         # Português
]

embeddings = model.encode(frases)

similaridade = util.cos_sim(embeddings, embeddings)
print("Similaridade entre frases em diferentes idiomas:")
print(similaridade)

Similaridade entre frases em diferentes idiomas:
tensor([[1.0000, 0.9863, 0.8716, 0.9882, 0.9871],
        [0.9863, 1.0000, 0.9012, 0.9902, 0.9903],
        [0.8716, 0.9012, 1.0000, 0.8931, 0.9077],
        [0.9882, 0.9902, 0.8931, 1.0000, 0.9887],
        [0.9871, 0.9903, 0.9077, 0.9887, 1.0000]])


Essa matriz de similiaridade mostra o quão próximo estão no espaço vetorial mesmo sendo palavras em idiomas distintos. A frase "How are you?" tem similaridade 1 com ela mesmo, o que faz sentido ja que são a mesma frase, ela também sem similaridade 0.9863 (aproximadamente 1) com "¿Cómo estás?" que é o mesmo sentido só que em espanhl, e assim por diante 

4. Image & Text-Models

São modelos de deep learning capazes de representar iamgens e textos em um espaço vetorial comum. Isso significa que tnato imagens quanto stextos são mapeados para a mesma "linguagem" matemática, possibilitando a comparação e a semelehança entre eles. Vamos utilizar a imagens de dois cachorros na neve

![Dois cachorros na neve](./img/two_dogs_in_snow.png)

In [39]:
from sentence_transformers import SentenceTransformer
from PIL import Image

model = SentenceTransformer("clip-ViT-B-32")

img_emb = model.encode(Image.open("img/two_dogs_in_snow.png"))

text_emb = model.encode(
    ["Two dogs in the snow", "A cat on a table", "A picture of London at night"]
)

similarity_scores = model.similarity(img_emb, text_emb)
print(similarity_scores)

tensor([[0.3069, 0.1010, 0.1086]])


Os INSTRUCTOR Models são uma categoria especial de modelos projetados para seguir instruções específicas durante o processo de criação de embeddings, tornando-os úteis em tarefas que exigem uma interpretação contextualizada e orientada. Diferente dos modelos normais de Sentence Transformers, os INSTRUCTOR Models são treinados com um enfoque em instruções dadas explicitamente, o que permite a geração de embeddings de acordo com o contexto e a tarefa definidas pela instrução.

In [41]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("hkunlp/instructor-large")
embeddings = model.encode(
    [
        "Dynamical Scalar Degree of Freedom in Horava-Lifshitz Gravity",
        "Comparison of Atmospheric Neutrino Flux Calculations at Low Energies",
        "Fermion Bags in the Massive Gross-Neveu Model",
        "QCD corrections to Associated t-tbar-H production at the Tevatron",
    ],
    prompt="Represent the Medicine sentence for clustering: ",
)
print(embeddings.shape)

(4, 768)


Os Instrucotr Models geram embeddings ajustados para uma determinada ação/frase. No nosso exemplo a instrução ou prompt é usada para orientar o modelo a gerar embeddings com um foco particular em "sentenças de medicina para agrupamento" (medicine sentence for clustering) — embora os textos sejam de temas variados e científicos. 

 prompt="Represent the Medicine sentence for clustering: " isso sugere que o modelo deve interpretar cada frase científica fornecida como se ela estivesse relacionada à área médica e criar embeddings para fins de “clustering” (agrupamento) nesse contexto mesmo que as frases fornecidas não sejam especificamente sobre medicina, o prompt instrui o modelo a gerar embeddings que talvez pudessem ser úteis para tarefas relacionadas à medicina ou ao agrupamento de informações científicas em um contexto médico.

 Essa abordagem de codificação adiciona um viés ao embedding, fazendo com que ele capture informações de maneira que seja útil para tarefas específicas — neste caso, para agrupamento no contexto de medicina.

Comparação entre Sentence Embeddings, Transformer sem usar sentece Embedding s e Similariade com Spacy

In [45]:
# Sentence Embedding
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('distiluse-base-multilingual-cased-v1')  #Modelo base nao funcionou legal pq funciona apenas em ingles

frases = ["O gato está no tapete.", "O felino está em cima do carpete."]
embeddings = model.encode(frases)

similarity = util.cos_sim(embeddings[0], embeddings[1])
print(f"Similaridade Sentence Embeddings: {similarity.item()}")

Similaridade Sentence Embeddings: 0.8263471126556396


In [46]:
# Transformers sem usar sentence Embeddings

from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

inputs = tokenizer(frases, return_tensors='pt', padding=True, truncation=True)
outputs = model(**inputs)

embeddings = torch.mean(outputs.last_hidden_state, dim=1) # Pooling: média das representações de tokens

similaridade = F.cosine_similarity(embeddings[0], embeddings[1], dim=0)
print(f"Similaridade Transformers: {similaridade.item()}")

Similaridade Transformers: 0.8272312879562378


In [ ]:
! pip install spacy
! python -m spacy download pt_core_news_md

In [53]:
#Spacy
import spacy

pln = spacy.load('pt_core_news_md')

doc1 = pln("O gato está no tapete.")
doc2 = pln("O felino está em cima do carpete.")

similarity = doc1.similarity(doc2)
print(f"Similaridade SpaCy: {similarity}")

Similaridade SpaCy: 0.8976613572535077


Considerar uma aplicação que utilize similaridade semântica em um determinado contexto, por exemplo, 
1.  correção automática de questões discursivas: gabarito x respostas dos alunos; 
2.  escolha de candidatos a uma determinada vaga: perfil desejado do candidato x perfil obtido nos dados do currículo lattes, linkedin, ou outra RSO; 
3.  analisar se uma pergunta já foi feita: pergunta do usuário x perguntas realizadas em um Q&A system; entre outros.

In [58]:
# Correção automática de questão discursiva

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

gabarito = "Aprendizado supervisionado é um método de machine learning onde o modelo é treinado com um conjunto de dados rotulado, ou seja, onde as entradas e saídas desejadas são fornecidas. O objetivo é que o modelo aprenda a mapear as entradas para as saídas corretas, podendo generalizar para novos dados."

respostas_alunos = [
    "Aprendizado supervisionado é um método de machine learning onde o modelo é treinado com um conjunto de dados rotulado, ou seja, onde as entradas e saídas desejadas são fornecidas. O objetivo é que o modelo aprenda a mapear as entradas para as saídas corretas, podendo generalizar para novos dados.",  # Resposta correta
    "Aprendizado supervisionado é quando um modelo é treinado com dados que possuem as respostas corretas, permitindo que ele aprenda a partir desses dados.",  # Resposta parcial
    "Aprendizado supervisionado é um tipo de machine learning em que o modelo decide por si mesmo o que deve aprender sem receber nenhum tipo de resposta ou dado rotulado.",  # Resposta incorreta
    "Aprendizado supervisionado é quando o professor fica supervisionando", #errada
    "2+ 2 = 4"
]

embedding_gabarito = model.encode(gabarito)
embeddings_respostas = model.encode(respostas_alunos)

similaridades = [util.cos_sim(embedding_gabarito, resposta).item() for resposta in embeddings_respostas]

notas = []
for similaridade in similaridades:
    if similaridade >= 0.85:
        nota = 1.0  
    elif 0.5 <= similaridade < 0.85:
        nota = 0.5  
    else:
        nota = 0.0  
    notas.append(nota)

for i, (resposta, similaridade, nota) in enumerate(zip(respostas_alunos, similaridades, notas)):
    print(f"Resposta do Aluno {i+1}: {resposta}\nSimilaridade: {similaridade:.4f}\nNota: {nota}\n")


Resposta do Aluno 1: Aprendizado supervisionado é um método de machine learning onde o modelo é treinado com um conjunto de dados rotulado, ou seja, onde as entradas e saídas desejadas são fornecidas. O objetivo é que o modelo aprenda a mapear as entradas para as saídas corretas, podendo generalizar para novos dados.
Similaridade: 1.0000
Nota: 1.0

Resposta do Aluno 2: Aprendizado supervisionado é quando um modelo é treinado com dados que possuem as respostas corretas, permitindo que ele aprenda a partir desses dados.
Similaridade: 0.7983
Nota: 0.5

Resposta do Aluno 3: Aprendizado supervisionado é um tipo de machine learning em que o modelo decide por si mesmo o que deve aprender sem receber nenhum tipo de resposta ou dado rotulado.
Similaridade: 0.8389
Nota: 0.5

Resposta do Aluno 4: Aprendizado supervisionado é quando o professor fica supervisionando
Similaridade: 0.6378
Nota: 0.5

Resposta do Aluno 5: 2+ 2 = 4
Similaridade: 0.1115
Nota: 0.0



In [61]:
# Sistema para match de perfil de linkedin
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")
perfil_desejado = "Procuramos um engenheiro de machine learning com experiência em Python, modelagem preditiva, e habilidades em cloud computing para desenvolvimento de soluções escaláveis."

perfis_candidatos = [
    "Engenheiro de machine learning com 5 anos de experiência em Python e desenvolvimento de modelos preditivos. Experiência em plataformas de computação em nuvem e implantação de soluções de machine learning.",  # Alta similaridade
    "Cientista de dados com experiência em Python e estatísticas, com conhecimento em modelos de machine learning. Experiência com dados na nuvem e análise preditiva.",  # Similaridade média
    "Desenvolvedor de software com foco em front-end, experiência em JavaScript, HTML e CSS. Interesse em aprender mais sobre machine learning." , # Baixa similaridade
    "Desenvolvedor front-end, html, css, javascript, angular, react" #Perfil sem similaridade
]

embedding_perfil_desejado = model.encode(perfil_desejado)
embeddings_candidatos = model.encode(perfis_candidatos)
similaridades = [util.cos_sim(embedding_perfil_desejado, embedding_candidato).item() for embedding_candidato in embeddings_candidatos]

indice_candidato_contratado = similaridades.index(max(similaridades))

for i, (perfil, similaridade) in enumerate(zip(perfis_candidatos, similaridades)):
    status = " (Perfil Contratado)" if i == indice_candidato_contratado else ""
    print(f"Perfil do Candidato {i+1}: {perfil}\nSimilaridade: {similaridade:.4f}{status}\n")

perfil_contratado = perfis_candidatos[indice_candidato_contratado]
print(f"Perfil Contratado:\n{perfil_contratado}")

Perfil do Candidato 1: Engenheiro de machine learning com 5 anos de experiência em Python e desenvolvimento de modelos preditivos. Experiência em plataformas de computação em nuvem e implantação de soluções de machine learning.
Similaridade: 0.7847 (Perfil Contratado)

Perfil do Candidato 2: Cientista de dados com experiência em Python e estatísticas, com conhecimento em modelos de machine learning. Experiência com dados na nuvem e análise preditiva.
Similaridade: 0.6264

Perfil do Candidato 3: Desenvolvedor de software com foco em front-end, experiência em JavaScript, HTML e CSS. Interesse em aprender mais sobre machine learning.
Similaridade: 0.5244

Perfil do Candidato 4: Desenvolvedor front-end, html, css, javascript, angular, react
Similaridade: 0.1096

Perfil Contratado:
Engenheiro de machine learning com 5 anos de experiência em Python e desenvolvimento de modelos preditivos. Experiência em plataformas de computação em nuvem e implantação de soluções de machine learning.


In [62]:
#Analisar se uma pergunta foi feita: pergunta do usuário x perguntas realizadas em um QA
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

pergunta_usuario = "Como posso resetar minha senha?"

perguntas_anteriores = [
    "Como posso recuperar minha senha esquecida?",
    "Quais métodos de pagamento são aceitos no sistema?",
    "Como alterar meu endereço cadastrado?",
    "Qual é o horário de atendimento ao cliente?",
    "Como fazer para redefinir a senha da minha conta?"
]

embedding_pergunta_usuario = model.encode(pergunta_usuario)
embeddings_perguntas_anteriores = model.encode(perguntas_anteriores)

similaridades = [util.cos_sim(embedding_pergunta_usuario, pergunta).item() for pergunta in embeddings_perguntas_anteriores]

limiar_similaridade = 0.75

pergunta_ja_feita = False
for i, similaridade in enumerate(similaridades):
    if similaridade >= limiar_similaridade:
        print(f"Pergunta semelhante encontrada: '{perguntas_anteriores[i]}'")
        print(f"Similaridade: {similaridade:.4f}\n")
        pergunta_ja_feita = True

if not pergunta_ja_feita:
    print("Pergunta inédita, adicione ao sistema de Q&A.")


Pergunta semelhante encontrada: 'Como posso recuperar minha senha esquecida?'
Similaridade: 0.7982

Pergunta semelhante encontrada: 'Como fazer para redefinir a senha da minha conta?'
Similaridade: 0.7546



Dois textos podem ser similares apesar de não ter uma única palavra em comum?
DICA: Utilize os métodos de similaridade semântica discutidos em sala de aula para
responder à pergunta, considerando textos pequenos (até 5 palavras) e textos
médios (até 400 palavras).

R= Sim, mostramos nos códigos acimas que o embedding é a representação matemática de um texto (palavra, frase ou texto-doc), assim é possível fazer calculos com essa representação, um desses calculos é justamente a diferença entre esses dois vetores, quando menor a diferença maios próximos esses vetores estão, isso mostra a similaridade, ou seja, quando mais próximo mais similar/relacionado sÃo esses dois textos, assim diferente de uma bsuca SQL, a busca por similaridade não querer que os textos tenham as mesmas pakavras, para que eles sejam similares precisam ter sentidos próximos, para isso podemos utilizar, por exemplos, sinônimos. Podemos demostrar com o texto do gato que colocamos no exemplo de busca semantica: 

In [71]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

frase_1 = model.encode("O gato está no telhado")
frase_2 = model.encode("Um felino está sobre o teto da casa")

similaridade = util.cos_sim(frase_1, frase_2)
print(f"Similaridade semântica: {similaridade.item()}")


Similaridade semântica: 0.8588588237762451


Agora vamos utilizar exemplos maiores

In [72]:
import nltk
from nltk.corpus import brown  
from sentence_transformers import SentenceTransformer, util


nltk.download("brown")

model = SentenceTransformer("all-MiniLM-L6-v2")

def calcular_similaridade(texto1, texto2):
    embedding1 = model.encode(texto1)
    embedding2 = model.encode(texto2)
    similaridade = util.cos_sim(embedding1, embedding2).item()
    return similaridade

texto_medio1 = " ".join(brown.words()[:400])
texto_medio2 = " ".join(brown.words()[500:900])
similaridade_medio = calcular_similaridade(texto_medio1, texto_medio2)


print(f"Texto 1 : {texto_medio1[:200]}...")  
print(f"Texto 2 : {texto_medio2[:200]}...") 
print(f"Similaridade entre textos: {similaridade_medio:.4f}")


[nltk_data] Downloading package brown to /Users/lailson/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Texto 1 : The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term-end present...
Texto 2 : under fire for its practices in the appointment of appraisers , guardians and administrators and the awarding of fees and compensation . Wards protected The jury said it found the court `` has incorpo...
Similaridade entre textos: 0.6323
